# Pandas for high-performance tables

- Pandas is the high-powered Python library used for data analysis. 
  - It is built on top of NumPy and is capable of handling large datasets efficiently. 
  - Pandas is also used for data visualization as it can plot data in a tabular or graphical form. 
- It will be the basis of GeoPandas, which we will use to look at vector data
  - For now, we will start with an ordinary table of data.


In [ ]:
# Do our imports
import pandas
import os

# Set our file paths (YOU SHOULD HAVE DOWNLOADED THIS FROM GOOGLE DRIVE!)
data_directory = '../../../../data'
food_prices_filename = 'world_monthly_food_prices.csv'
food_prices_path = os.path.join(data_directory, food_prices_filename)

# Pandas provides a read_csv function that will read a CSV file into a dataframe
food_prices = pandas.read_csv(food_prices_path)

# Print the dataframe using default printing options
print(food_prices)

### Printing the dataframe head()

- It's a little hard to take in all the data because it skips rows and columns to make it fit
- An alternate way to view it is to use the `head()` method to see the first few rows of the table

In [ ]:
# Print the head of the dataframe. Reminder, the . operator is used to 
# access a function (technically called method) of an object. 
# Before I print it, I'm also going to change the default print settings to be wider

# Set the display width to 1000
pandas.set_option('display.width', 1000)

# Print it with the new settings
print(food_prices.head())

### Explore the different object attributes of the dataframe

- For instance look at the column names

In [ ]:
print('List of column names:', food_prices.columns)

### Get a specific column
- To get a column, use square braces with the name of the column.

In [ ]:
food_prices_value_column = food_prices['Value']

print('Specific column:', food_prices_value_column)

In [ ]:
print('Specific value in that column:', food_prices_value_column[6])

### Iterate through columns

- The `food_prices.columns` attribute can be made into a list for iteration
- Other (better as we will eventually see) ways to iterate through columns are possible
  - Such as itertools
- For now, iterate through the columns and sum them up.

In [ ]:
# The columns attribute is not quite a list. It's a panda's Series and has information about what it indexes
print(food_prices.columns)

In [ ]:
# But we can make it into an ordinary list
columns_list = list(food_prices.columns)
print('columns_list', columns_list)

In [ ]:
# Iterate through the columns and print the sum
for column_name in columns_list:
    print('Currently accessing column', column_name)
    
    current_column = food_prices[column_name]
    
    # Check the datatype of the current_column
    current_data_type = current_column.dtype
    print('Datatype of current_column:', current_data_type)
    
    if current_data_type == 'float64':
        print('    Sum of column', column_name, 'is', current_column.sum())

### Under the hood

- All parts of the pandas dataframe are represented via numpy arrays
  - If you want to access them as numpy arrays instead of Pandas dataframes or columns, you can use the .values attribute

In [ ]:
numpy_array = food_prices_value_column.values
print(numpy_array)

### Default plotting

- By default, pandas has a plot function
  - Let's try it

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
plt.plot(food_prices['Value'])
plt.show() 

### Discussion point

- Why is the data like this?

### Pandas Functionality

- Let's start by importing everything afresh and setting a random seed.
  - Setting the random seed means we will get the same random numbers each time we run the code.

In [ ]:
import os

import numpy as np
import pandas as pd

# Set a seed value for the random generator
np.random.seed(48151623)

### Create an individual column

- In pandas, they are called Series
- One very important thing to note is that in the output, we see TWO columns
  - Pandas always includes an index for every Series or Dataframe 
  - The index is the leftmost column, and is used to identify each row

In [ ]:
# Creating a Series by passing a list of values, letting pandas create a default integer index:
s = pd.Series([1, 3, 5, np.nan, 6, 8])
print(s)

### There are many built-in ways of generating series

- For instance, you can automatically generate a series of dates

In [ ]:
# Pandas is very detailed in dealing with dates and all the quirks (leap year?) that this leads to.
dates = pd.date_range('20130101', periods=6)
print(dates)

### Making new dataframes

- Dataframes can be made from a a dictionary of series, or from a list of lists, or from a numpy array, or several other possiblities.
  - We'll explore a few here
  - The primary way is to call the pd.DataFrame() class with specific arguments
    - Returns a new dataframe object here named `df`

In [ ]:
# Creating a DataFrame by passing a NumPy array, with a datetime index and labeled columns:
df_rand = pd.DataFrame(np.random.randn(6, 4), columns=['A', 'B', 'C', 'D'])
print('df:\n', df_rand)

In [ ]:
# A mre complex example
df2 = pd.DataFrame({'A': 1.,
                    'B': pd.Timestamp('20130102'),
                    'C': pd.Series(1, index=list(range(4)), dtype='float32'),
                    'D': np.array([3] * 4, dtype='int32'),
                    'E': pd.Categorical(["test", "train", "test", "train"]),
                    'F': 'foo'})
print(df2.head())

In [ ]:
# In addition to printing the head, there is also a handy .describe() method
# this gives descriptive statistics.
df2.describe()

### It's numpy matrices all the way down

- Dataframes add extra information to make it feel like a table or spreadsheet that we're used to

In [ ]:
# Another way you can get back to the the raw array is to_numpy()
# (though this might lose a lot of functionality).
a = df_rand.to_numpy()
print('a\n', a)

### Operations on dataframes: sorting

- Here we are going to use the fast sorting algorithm provided by Pandas
  - It's called `sort_values()`
  - I want to illustrate the most common mistake people make with Pandas and discuss it.

In [ ]:
# We're going to sort by B. But let's print out the B column first.
print('B column:\n', df_rand['B'])

# Now let's call the sort_values method on the dataframe.
df_rand.sort_values(by='B')

# Print it out to compare
print('Sorted:\n', df_rand['B'])

### Why did it not chamge?

- The reason is that `sort_values()` returns a new dataframe, but we didn't assign it to anything.
  - We can assign it to a new dataframe, or we can use the `inplace=True` argument to change the dataframe in place.

In [ ]:
# Easy way to get around this is just to assign the returned dataframe to a variable (even the input variable)
df_rand = df_rand.sort_values(by='B')
print('Sorted with return:\n', df_rand)

# Alternatively, if you hate returning things, there is the inplace=True command, which will modify the df ... inplace.
df_rand.sort_values(by='B', inplace=True)
print('Sorted inplace:\n', df_rand)

### Slicing and Dicing

- We can slice and dice dataframes in many ways
- - We can use the `[]` operator
  - We can use the `loc` and `iloc` methods
    - loc is for accessing via labels/indices on the data
    - iloc is for accessing by position of the data


In [ ]:
# Slicing with the [] works like numpy

# Starts with rows
print(df_rand[0:3])

In [ ]:
# You can get unintuitive results using the standard 
# Python / Numpy expressions for selecting and setting are intuitiveits best to use
# the optimized pandas data access methods, .at, .iat, .loc and .iloc.

## Selecting by LABELS, loc and iloc

r = df_rand.loc[0] # 0-th row.
print(r)

In [ ]:
# Discuss difference between df['A'] and df.loc[0]
r = df_rand.loc[0, 'A']
print(r)

In [ ]:
r = df_rand.loc[:, 'A'] # Colon is a slice, an empty colon means ALL the values.
print(r)

In [ ]:
# SELECTING BY POSITION
r = df_rand.iloc[3]
print(r)

In [ ]:
# Selecting with slices
r = df_rand.iloc[3:5, 0:2]
print(r)

In [ ]:
# Slices again with an empty slice.
r = df_rand.iloc[1:3, :]
print(r)

In [ ]:
r = df_rand.iloc[:, 1:3]
print(r)

In [ ]:
# Boolean indexing
# Using a single column’s values to select data.
r = df_rand[df_rand['A'] > 0]
print(r)

In [ ]:
# Make a copy (why?) and add a column
df_copy = df_rand.copy()
df_copy['E'] = ['one', 'one', 'two', 'three', 'four', 'three']
r = df_copy[df_copy['E'].isin(['two', 'four'])]
print(r)

In [ ]:
# Setting by assigning with a NumPy array:
df = df_rand.copy()
df.loc[:, 'D'] = np.array([5] * len(df))

### Merging and Joining

- Merging data is one of the most time-intensive and important data-cleaning skills
  - Lets you create "new" data by merging other old data!


In [ ]:
# Join
# SQL style merges. See the Database style joining section.

left = pd.DataFrame({'key': ['foo', 'bar'], 'lval': [1, 2]})
right = pd.DataFrame({'key': ['foo', 'bar'], 'rval': [4, 5]})

print(left)
print(right)

df = pd.merge(left, right, on='key')
print('df:\n', df)

In [ ]:
# Stacking
stacked = df.stack()
# print('stacked:\n', stacked)


# Pivot Tables
df = pd.DataFrame({'A': ['one', 'one', 'two', 'three'] * 3,
                   'B': ['A', 'B', 'C'] * 4,
                   'C': ['foo', 'foo', 'foo', 'bar', 'bar', 'bar'] * 2,
                   'D': np.random.randn(12),
                   'E': np.random.randn(12)})

# print(df) # SPREADSHEET VIEW
df = pd.pivot_table(df, values='D', index=['A', 'B'], columns=['C'])
# print(df) # Multiindexed (Pivot table) view.

# NOTICE that a pivot table is just the above date but where specific things have been made into multi-level
# indices.

# PLOTTING
ts = pd.Series(np.random.randn(1000),
            index=pd.date_range('1/1/2000', periods=1000))

ts = ts.cumsum()
ts.plot()

In [ ]:
# Writing to files

df.to_csv('foo.csv')

# Appendix: Additional tasks with Pandas


In [ ]:
# Missing data

# First we're going to create a new df by "reindexing" the old one, which will shuffle the data into a new
# order according to the index provided. At the same time, we're going to add on a new, empty column
# EE, which we set as 1 for the first two obs.

df1 = df.reindex(index=[2, 0, 1, 3], columns=list(df.columns) + ['EE'])
df1.loc[0:1, 'EE'] = 1
print(df1)

In [ ]:
# Apply: Similar to R. Applies a function across many cells (fast because it's vectorized)
df.apply(np.cumsum)
df.apply(lambda x: x.max() - x.min())
print(df)

In [ ]:
# Concat
s = pd.Series(range(0, 6))
# print('s', s)

r = pd.concat([df, s]) # Concatenate it, default is by row, which just puts it on the bottom.

r = pd.concat([df, s], axis=1) # Concatenate as a new column

# print(r) # Result when concatenating a series of the same size.

s = pd.Series(range(0, 7))
r = pd.concat([df, s], axis=1) # Concatenate as a new column

s = pd.Series(range(0, 2))
r = pd.concat([df, s], axis=1) # Concatenate as a new column